# Experiment 1:

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, TensorDataset


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/CA_Repo')
from gm_utils_v2 import gradient_matching, get_network, get_daparam, evaluate_synset, save_image, match_loss, get_dataset, train_and_test_model
from helpers import plot_gradient_loss_by_iteration_by_class_and_total

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
class Args:
    method = "DC"
    dataset = "MNIST"
    model =  "ConvNet"
    ipc = 1  # Images per class
    num_exp = 1
    num_eval = 3  # Number of evaluations per experiment
    Iteration = 25  # Training iterations
    lr_img = 0.1  # Learning rate for synthetic images
    lr_net = 0.01  # Learning rate for network
    batch_real = 256  # Batch size for real data
    batch_train = 128  # Batch size for synthetic data training
    init = "noise"  # Initialize synthetic data from noise
    hessian = False
    eval_mode = "S"
    data_path = "./data"
    save_path = "./results_t_1"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    epoch_eval_train = 50  # Number of epochs to train evaluation models
    outer_loop = 10  # Number of outer-loop updates for synthetic data\
    dis_metric = 'ours'

### ConvNet

In [ ]:
args = Args()
eval_it_pool = [0, 5, 10, 15, 20, args.Iteration]
args.model = "ConvNet"
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)

Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 20:29:32] Training begins
Iteration# 0
Evaluating model ConvNet at iteration 0...
Image grid saved to ./results_t_1/vis_DC_MNIST_ConvNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 23017.2157
Iteration# 1
Iteration 1: Total Gradient Loss = 15907.1545
Iteration# 2
Iteration 2: Total Gradient Loss = 13426.1968
Iteration# 3
Iteration 3: Total Gradient Loss = 11822.5774
Iteration# 4
Iteration 4: Total Gradient Loss = 11026.4324
Iteration# 5
Evaluating model ConvNet at iteration 5...
Image grid saved to ./results_t_1/vis_DC_MNIST_ConvNet_1ipc_single_iter5.png
Iteration 5: Total Gradient Loss = 10274.6848
Iteration# 6
Iteration 6: Total Gradient Loss = 9277.0974
Iteration# 7
Iteration 7: Total

In [ ]:
accs_all_exps

{'ConvNet': [0.0496,
  0.1346,
  0.1084,
  0.6638,
  0.7074,
  0.6873,
  0.6969,
  0.6398,
  0.6551,
  0.7274,
  0.7385,
  0.7251,
  0.6926,
  0.6341,
  0.7286,
  0.7191,
  0.724,
  0.7224]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training ConvNet on the synthetic data
Epoch 1/10, Loss: 2.3829
Epoch 2/10, Loss: 1.9108
Epoch 3/10, Loss: 1.3883
Epoch 4/10, Loss: 1.0729
Epoch 5/10, Loss: 0.8029
Epoch 6/10, Loss: 0.5900
Epoch 7/10, Loss: 0.4198
Epoch 8/10, Loss: 0.3022
Epoch 9/10, Loss: 0.2183
Epoch 10/10, Loss: 0.1568
Accuracy on test set (trained on synthetic data): 0.8365


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.model + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/ConvNet__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/ConvNet__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/ConvNet__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/ConvNet__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/ConvNet__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/ConvNet__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/ConvNet__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/ConvNet__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/ConvNet__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/ConvNet__class9.png
Total gradient loss by iteration plot saved to ./results/ConvNet__total.png


### MLP

In [ ]:
args = Args()
args.model = "MLP"
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 21:46:14] Training begins
Iteration# 0
Evaluating model MLP at iteration 0...
Image grid saved to ./results_t_1/vis_DC_MNIST_MLP_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 22151.8657
Iteration# 1
Iteration 1: Total Gradient Loss = 20874.1073
Iteration# 2
Iteration 2: Total Gradient Loss = 20202.7135
Iteration# 3
Iteration 3: Total Gradient Loss = 19991.1215
Iteration# 4
Iteration 4: Total Gradient Loss = 19319.4038
Iteration# 5
Evaluating model MLP at iteration 5...
Image grid saved to ./results_t_1/vis_DC_MNIST_MLP_1ipc_single_iter5.png
Iteration 5: Total Gradient Loss = 19381.9164
Iteration# 6
Iteration 6: Total Gradient Loss = 18543.9957
Iteration# 7
Iteration 7: Total Gradient Loss 

In [ ]:
accs_all_exps

{'MLP': [0.1702,
  0.1094,
  0.1351,
  0.2515,
  0.3078,
  0.2067,
  0.4403,
  0.5381,
  0.4225,
  0.6391,
  0.6024,
  0.5918,
  0.5138,
  0.5262,
  0.6169,
  0.4117,
  0.4513,
  0.4547]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training MLP on the synthetic data
Epoch 1/10, Loss: 2.2809
Epoch 2/10, Loss: 1.7359
Epoch 3/10, Loss: 1.5078
Epoch 4/10, Loss: 1.3576
Epoch 5/10, Loss: 1.2330
Epoch 6/10, Loss: 1.1153
Epoch 7/10, Loss: 0.9991
Epoch 8/10, Loss: 0.8870
Epoch 9/10, Loss: 0.7737
Epoch 10/10, Loss: 0.6608
Accuracy on test set (trained on synthetic data): 0.4522


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.model + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/MLP__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/MLP__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/MLP__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/MLP__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/MLP__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/MLP__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/MLP__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/MLP__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/MLP__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/MLP__class9.png
Total gradient loss by iteration plot saved to ./results/MLP__total.png


### LeNet

In [ ]:
args = Args()
args.model = "LeNet"
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)


In [ ]:
accs_all_exps, data_save, gradient_losses, class_gradient_losses = gradient_matching(
    args=args,
    dst_train=dst_train,
    num_classes=num_classes,
    im_size=im_size,
    channel=channel,
    std=std,
    mean=mean,
    eval_it_pool = eval_it_pool,
    testloader=testloader,
    get_network=get_network,
    evaluate_synset=evaluate_synset,
    save_image=save_image,
    match_loss=match_loss,
    get_daparam=get_daparam
)


Class distribution:
Class 0: 5923 images
Class 1: 6742 images
Class 2: 5958 images
Class 3: 6131 images
Class 4: 5842 images
Class 5: 5421 images
Class 6: 5918 images
Class 7: 6265 images
Class 8: 5851 images
Class 9: 5949 images
Initializing synthetic data from random noise...
[2024-12-16 21:12:30] Training begins
Iteration# 0
Evaluating model LeNet at iteration 0...
Image grid saved to ./results_t_1/vis_DC_MNIST_LeNet_1ipc_single_iter0.png
Iteration 0: Total Gradient Loss = 14407.3496
Iteration# 1
Iteration 1: Total Gradient Loss = 15169.9595
Iteration# 2
Iteration 2: Total Gradient Loss = 15313.4678
Iteration# 3
Iteration 3: Total Gradient Loss = 13798.4962
Iteration# 4
Iteration 4: Total Gradient Loss = 13542.4274
Iteration# 5
Evaluating model LeNet at iteration 5...
Image grid saved to ./results_t_1/vis_DC_MNIST_LeNet_1ipc_single_iter5.png
Iteration 5: Total Gradient Loss = 14552.1393
Iteration# 6
Iteration 6: Total Gradient Loss = 14732.7352
Iteration# 7
Iteration 7: Total Gradie

In [ ]:
accs_all_exps

{'LeNet': [0.1726,
  0.0739,
  0.0346,
  0.3201,
  0.2149,
  0.3965,
  0.4925,
  0.4349,
  0.4431,
  0.5804,
  0.554,
  0.6262,
  0.6227,
  0.5501,
  0.6442,
  0.6424,
  0.6701,
  0.555]}

In [ ]:
# Train and test on synthetic data
test_loader_real = testloader
print(f"Training {args.model} on the synthetic data")
synthetic_images, synthetic_labels = data_save[0]  # Get the first experiment's synthetic data
synthetic_dataset = TensorDataset(synthetic_images, synthetic_labels)
train_loader_synthetic = DataLoader(synthetic_dataset, batch_size=64, shuffle=True)
synthetic_model = get_network(args.model, channel, num_classes, im_size)
synthetic_accuracy = train_and_test_model(synthetic_model, train_loader_synthetic, test_loader_real, num_epochs=10, device=args.device)
print(f"Accuracy on test set (trained on synthetic data): {synthetic_accuracy:.4f}")


Training LeNet on the synthetic data
Epoch 1/10, Loss: 2.3030
Epoch 2/10, Loss: 2.2666
Epoch 3/10, Loss: 2.2363
Epoch 4/10, Loss: 2.2042
Epoch 5/10, Loss: 2.1664
Epoch 6/10, Loss: 2.1218
Epoch 7/10, Loss: 2.0703
Epoch 8/10, Loss: 2.0110
Epoch 9/10, Loss: 1.9437
Epoch 10/10, Loss: 1.8693
Accuracy on test set (trained on synthetic data): 0.4621


In [ ]:
plot_gradient_loss_by_iteration_by_class_and_total(class_gradient_losses, gradient_losses, save_path="./results/" + args.model + "_" )

Gradient loss by iteration plot for Class 0 saved to ./results/LeNet__class0.png
Gradient loss by iteration plot for Class 1 saved to ./results/LeNet__class1.png
Gradient loss by iteration plot for Class 2 saved to ./results/LeNet__class2.png
Gradient loss by iteration plot for Class 3 saved to ./results/LeNet__class3.png
Gradient loss by iteration plot for Class 4 saved to ./results/LeNet__class4.png
Gradient loss by iteration plot for Class 5 saved to ./results/LeNet__class5.png
Gradient loss by iteration plot for Class 6 saved to ./results/LeNet__class6.png
Gradient loss by iteration plot for Class 7 saved to ./results/LeNet__class7.png
Gradient loss by iteration plot for Class 8 saved to ./results/LeNet__class8.png
Gradient loss by iteration plot for Class 9 saved to ./results/LeNet__class9.png
Total gradient loss by iteration plot saved to ./results/LeNet__total.png
